# Импортируем библиотеки

In [ ]:
import pandas as pd
import re

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor

import xgboost as xgb
import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session

np.random.seed(42)

# Предобработка данных

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

Количество пропущенных значенний:

In [ ]:
df.isna().sum() / (df.shape[0])*100 #процент пропусков

Удаляем строки без целевой переменной:

In [ ]:
df = df.loc[df['target'].isna() == False]

In [ ]:
df.shape

Очищаем целевую переменную:

In [ ]:
def target_format(target):
    target = re.sub('[^0-9]', '', target) #оставляем только цифры
    target = int(target)
    return target

In [ ]:
df['target'] = df['target'].apply(target_format)
df['target'].describe()

Очищаем признаки:

In [ ]:
# Преобразовываем в действительные числа
def features_float_format(feature):
    if feature == 0:
        return feature
    feature = re.sub('1 1/2', '1.5', feature) # заменяем 1 1/2 на 1.5 (признак stories)
    feature = re.sub('[^0-9,\.]', '', feature) # оставляем только цифры
    feature = re.sub(',', '.', feature) # заменяем ',' на '.'
    
    try:
        feature = float(feature)
    #если feature - пустая строка, возвращаем 0
    except:
        feature = 0
    return feature
# Преобразовываем статус
def status_format(status):
    status = status.lower()
    # если дом ещё не сделан,оставляем только 'coming soon', дату удаляем
    if status.startswith('coming soon'): 
        status = 'coming soon'
        
    status = re.sub('[^a-z]', ' ', status) # оставляем только буквы
    status = re.sub(r'\b\w{,2}\b', '', status) # удаляем сочетание из 1 и 2 букв
    status = re.sub(r'\s+', '', status) # заменяем 1 или более пробелов на ''
    if status == 'active' or status == 'for sale': # бóльшая часть домов - for sale или active
        status = 1
    else:
        status = 0
    return status
#Преобразовываем фичу камина
def fireplace_format(fireplace):
    if fireplace == 0:
        return fireplace
    fireplace = fireplace.lower()
    
    if fireplace.count('no')>0:
        fireplace = 0
    else:
        fireplace = 1
    return fireplace

In [ ]:
dummy_features = ['status','state']  
drop_features = ['street', 'mls-id', 'MlsId', 'schools', 'homeFacts', 'city', 'zipcode'] 

#### Status

In [ ]:
df.status.value_counts()

Приблизительно 80% домов - ```for sale``` или ```for sale```, поэтому вместо них ставим ```1```, иначе ```0```
(заполняем пропуски на пустые строки, и применяем ```status_format```):

In [ ]:
df['status'] = df['status'].fillna('')
df['status'] = df['status'].apply(status_format)

#### Property Type

In [ ]:
df.propertyType.value_counts()

Приблизительно 50% домов - ```single family```, поэтому оставляем только этот признак:

In [ ]:
def propertyType_format(propertyType):
    propertyType = propertyType.lower()
    propertyType = re.sub('[^a-z]', ' ', propertyType)# оставляем только буквы
    
    # если начинается на 'single family' - ставим 1, иначе 0
    if propertyType.startswith('single family'): 
        propertyType = 1 
    else:
        propertyType = 0
    return propertyType

In [ ]:
df['propertyType'] = df['propertyType'].fillna('')
df['propertyType'] = df['propertyType'].apply(propertyType_format)

#### Private pool
Вместо пропусков ставим ```no```, заполняем вместо ```yes``` - 1, вместо ```no``` - 0

In [ ]:
df['private pool'] = df['private pool'].fillna('no')
df['private pool'] = df['private pool'].apply(lambda pool: 1 if pool.lower() == 'yes' else 0)

df['PrivatePool'] = df['PrivatePool'].fillna('no')
df['PrivatePool'] = df['PrivatePool'].apply(lambda pool: 1 if pool.lower() == 'yes' else 0)

df['PrivatePool'] = df['private pool'] | df['PrivatePool']
df.drop(['private pool'], axis=1, inplace=True)

#### Baths, sqft, beds, stories, fireplace
Заполняем пропуски нулями
К ```Baths, sqft, beds, stories``` применяем ```features_float_format```, а к ```fireplace``` - ```fireplace_format```:

In [ ]:
df['baths'] = df['baths'].fillna(0)
df['baths'] = df['baths'].apply(features_float_format)

df['sqft'] = df['sqft'].fillna(0)
df['sqft'] = df['sqft'].apply(features_float_format)

df['beds'] = df['beds'].fillna(0)
df['beds'] = df['beds'].apply(features_float_format)

df['stories'] = df['stories'].fillna(0)
df['stories'] = df['stories'].apply(features_float_format)

df['fireplace'] = df['fireplace'].fillna(0)
df['fireplace'] = df['fireplace'].apply(fireplace_format)

#### Year Built

In [ ]:
df['homeFacts'] = df['homeFacts'].apply(eval) # конвертируем строку в словарь, используя eval

Нужное нам значение хранится в первом словаре, который хранится в списке, который, в свою очередь, хранится в словаре с ключом ```atAGlanceFacts```:

In [ ]:
M = df['homeFacts'][0]
print(M,'\n')
print(M['atAGlanceFacts'],'\n')
print(M['atAGlanceFacts'][0],'\n')
print(M['atAGlanceFacts'][0]['factValue'],'\n')

In [ ]:
def homeFacts_format(homeFacts):
    # если homeFacts - непустой словарь, берём значение 'atAGlanceFacts', иначе 0
    homeFacts = homeFacts.get('atAGlanceFacts', 0) 
    if homeFacts == 0: 
        return 0
    # берём первый елемент списка
    homeFacts = homeFacts[0]
    # если словарь не пуст, то берем значение
    if homeFacts.get('factLabel') == 'Year built':
        homeFacts = homeFacts.get('factValue')
    # если homeFacts - число, то выводим его, иначе 0
    try:
        homeFacts = int(homeFacts)
    except:
        homeFacts = 0
    return homeFacts

In [ ]:
df['yearBuilt'] = df['homeFacts'].apply(homeFacts_format)

#### Schools rating

In [ ]:
df['schools'] = df['schools'].apply(eval) # конвертируем строку в словарь, используя eval

Список с рейтингами школ хранится в словаре, который хранится в списке:

In [ ]:
M = df['schools'][0]
print(M,'\n')
print(M[0],'\n')
print(M[0]['rating'])

In [ ]:
def rating_format(schools):
    # если schools - непустой словарь, берём значение 'rating', иначе 0
    schools = schools[0]
    schools = schools.get('rating', 0)
    if schools == 0: 
        return 0
    
    rating = []
    for x in schools:
        x = re.sub('/10','',x) # если рейтинг записан в формате 'x/10', заменяем на 'x'
        rating.append(features_float_format(x)) # с помощью features_float_format превращаем x в число
    # ищем среднее арифметическое рейтингов
    return np.mean(rating)

Применяем ```rating_format```, если результат - ```nan```, заменяем на ```0```:

In [ ]:
df['rating'] = df['schools'].apply(rating_format)
df['rating'] = df['rating'].fillna(0)

#### Distance
По аналогии с ```rating```, выбираем значение из списка словарей и ищем среднее арифметическое расстояний

In [ ]:
def distance_format(schools):
    # если schools - непустой словарь, берём значение 'data', иначе 0
    schools = schools[0]
    schools = schools.get('data', 0)
    if schools == 0: 
        return 0
    # если schools - непустой словарь, берём значение 'distance', иначе 0
    schools = schools.get('Distance', 0)
    if schools == 0: 
        return 0
    
    distance=[]
    for x in schools:
        distance.append(features_float_format(x)) # с помощью features_float_format превращаем x в число
    # ищем среднее арифметическое расстояний
    return np.mean(distance)

Применяем ```distance_format``` и заполняем пропуски нулями:

In [ ]:
df['distance'] = df['schools'].apply(distance_format)
df['distance'] = df['distance'].fillna(0)

Удаляем ненужные фичи:

In [ ]:
df.drop(drop_features, axis=1, inplace=True)

#### State 
Штат

In [ ]:
df = pd.concat([df, pd.get_dummies(df['state'])], axis=1)
df.drop('state', axis=1, inplace=True)

# Статистический анализ

In [ ]:
df.head()

In [ ]:
df.describe(include='all')

In [ ]:
corr = df.corr()

cmap = sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "5pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "8pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '8pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [ ]:
_ = plt.hist(np.log(df['target']+1),bins = 'auto')
plt.title('log of target')
plt.show

In [ ]:
_ = plt.hist(np.log(df['sqft']+1),bins = 20)
plt.title('log of area')
plt.show()

In [ ]:
_ = plt.hist(np.log(df['beds']+1),bins = 25)
plt.title('log of beds')
plt.show()

In [ ]:
_ = plt.hist(np.log(df['stories']+1),bins = 15)
plt.title('log of stories')
plt.show()

In [ ]:
_ = plt.hist(df['rating'],bins = 10)
plt.title('rating')
plt.show()

In [ ]:
_ = plt.hist(np.log(df['distance']+1),bins = 'auto')
plt.title('log of distance')
plt.show()

In [ ]:
corr['target'].sort_values(ascending = False)[:10]

In [ ]:
_ = plt.scatter(df['rating'],df['target'])

plt.title('rating/target')
plt.xlabel('rating')
plt.ylabel('target')

plt.show()

In [ ]:
_ = plt.scatter(df['distance'],df['target'])

plt.xlim(-2,80)
plt.ylim(-100,1e8)

plt.title('distance/target')
plt.xlabel('distance')
plt.ylabel('target')

plt.show()

In [ ]:
_ = plt.scatter(df['stories'],df['target'])
plt.xlim(-5,105)
plt.ylim(-10,1e7*8)

plt.title('stories/target')
plt.xlabel('stories')
plt.ylabel('target')

plt.show()

In [ ]:
_ = plt.scatter(df['beds'],df['target'])

plt.xlim(-5,105)
plt.ylim(-10,1e7*8)

plt.title('beds/target')
plt.xlabel('beds')
plt.ylabel('target')

plt.show()

In [ ]:
_ = plt.scatter(df['yearBuilt'],df['target'])

plt.xlim(1650,2050)
plt.ylim(-50,8*1e7)

plt.title('yearBuilt/target')
plt.xlabel('yearBuilt')
plt.ylabel('target')

plt.show()

In [ ]:
_ = plt.scatter(df['sqft'],df['target'])

plt.xlim(500,0.25*1e5)
plt.ylim(-1000,3*1e7)

plt.title('area/target')
plt.xlabel('area')
plt.ylabel('target')

plt.show()

# Построение моделей

Разобьем данные на обучающаю и тестовую выборки:

In [ ]:
X, y = df.drop('target', axis = 1), df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

Сделаем функцию для оценки точности предсказаний:

In [ ]:
def evaluate(clf, X_train, y_train, X_test, y_test):
    
    train_predict = clf.predict(X_train)
    test_predict = clf.predict(X_test)
    
    print('train mse :', mean_squared_error(y_train, train_predict) )
    print('test mse  :', mean_squared_error(y_test, test_predict) )
    
    print('train rmse :', np.sqrt(mean_squared_error(y_train, train_predict) ))
    print('test rmse  :',np.sqrt(mean_squared_error(y_test, test_predict) ))
    #Coefficient of determination
    print('train R^2', clf.score(X_train,y_train)) 
    print('test R^2', clf.score(X_test,y_test))

### Линейная регрессия

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
evaluate(lr,X_train, y_train, X_test, y_test)

Коэффициент детерминации ниже 0.1, а значит модель очень плохо предсказывает результат. Попробуем различные деревья решений:

### Деревья решений

С помощью ```optune``` переберём различные варианты ```max_depth```:

In [ ]:
def objective(trial):
    tree_max_depth = trial.suggest_int('tree_max_depth', 5, 10)
    
    tree = DecisionTreeRegressor(max_depth  = tree_max_depth, random_state = 42)
    tree.fit(X_train,y_train)

    test_predict = tree.predict(X_test)

    return mean_squared_error(y_test, test_predict)

study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 7)

Итог: при максимальной глубине меньше 7 модель недообучается, а при максимальной глубине больше 7 - переобучается. 

In [ ]:
tree = DecisionTreeRegressor(max_depth  = 7 ,random_state = 42)
tree.fit(X_train,y_train)

In [ ]:
evaluate(tree,X_train, y_train, X_test, y_test)

### Попробуем различные ансамбли:

С помощью ```optune``` переберём различные варианты ```n_estimators```:

In [ ]:
def objective(trial):
    bagging_n_estimators = trial.suggest_int("rf_n_estimators", 5, 19)
    
    bagging_trees = BaggingRegressor(tree, n_estimators = bagging_n_estimators)
    bagging_trees.fit(X_train,y_train)

    test_predict = bagging_trees.predict(X_test)

    return mean_squared_error(y_test, test_predict)

study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 7)

In [ ]:
bagging_trees = BaggingRegressor(tree, n_estimators = 19)
bagging_trees.fit(X_train,y_train)

In [ ]:
evaluate(bagging_trees,X_train, y_train, X_test, y_test)

С помощью ```optune``` переберём различные варианты ```n_estimators``` и  ```max_depth```:

In [ ]:
def objective(trial):
    rf_max_depth = trial.suggest_int('rf_max_depth', 5, 10)
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 8, 13)
    
    random_forest = RandomForestRegressor(n_estimators = rf_n_estimators, max_depth = rf_max_depth, random_state = 42)
    random_forest.fit(X_train,y_train)

    test_predict = random_forest.predict(X_test)

    return mean_squared_error(y_test, test_predict)

study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 8)

In [ ]:
random_forest = RandomForestRegressor(n_estimators = 13, max_depth = 10, random_state = 42)
random_forest.fit(X_train,y_train)

In [ ]:
evaluate(random_forest,X_train, y_train, X_test, y_test)

С помощью ```optune``` переберём различные варианты ```n_estimators``` :

In [ ]:
def objective(trial):
    ada_n_estimators = trial.suggest_int("ada_n_estimators", 5, 10)
    
    adaboost = AdaBoostRegressor(tree,n_estimators = ada_n_estimators, random_state = 42)
    adaboost.fit(X_train,y_train)

    test_predict = adaboost.predict(X_test)

    return mean_squared_error(y_test, test_predict)

study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 6)

In [ ]:
adaboost = AdaBoostRegressor(tree,n_estimators = 6, random_state = 42)
adaboost.fit(X_train,y_train)

In [ ]:
evaluate(adaboost,X_train, y_train, X_test, y_test)

С помощью ```optune``` переберём различные варианты ```n_estimators, max_depth, min_samples_split```:

In [ ]:
def objective(trial):
    gb_max_depth = trial.suggest_int('gb_max_depth', 5, 10)
    gb_n_estimators = trial.suggest_int('gb_n_estimators', 8, 20)
    gb_min_samples_split = trial.suggest_int('gb_min_samples_split', 2, 5)
    
    gradientboosting = GradientBoostingRegressor(n_estimators = gb_n_estimators, 
                                                 max_depth = gb_max_depth,
                                                 min_samples_split = gb_min_samples_split,
                                                 loss = 'ls')

    gradientboosting.fit(X_train, y_train)
    test_predict = gradientboosting.predict(X_test)

    return mean_squared_error(y_test, test_predict)

study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 12)

In [ ]:
gradientboosting = GradientBoostingRegressor(n_estimators = 20, max_depth = 12, min_samples_split = 4,loss = 'ls')
gradientboosting.fit(X_train, y_train)

In [ ]:
evaluate(gradientboosting,X_train, y_train, X_test, y_test)

#### XGBoost

С помощью ```optune``` переберём различные варианты ```n_estimators, max_depth```:

In [ ]:
def objective(trial):
    xgb_max_depth = trial.suggest_int('xgb_max_depth', 5, 10)
    xgb_n_estimators = trial.suggest_int('xgb_n_estimators', 8, 125)
    model = xgb.XGBRegressor(max_depth = xgb_max_depth, 
                                 n_estimators = xgb_n_estimators,
                                 seed = 42) 

    model.fit(X_train, y_train)
    test_predict = model.predict(X_test)

    return mean_squared_error(y_test, test_predict)

study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 12)

In [ ]:
model = xgb.XGBRegressor(max_depth = 6,
                                 n_estimators = 90,
                                 seed = 42) 

In [ ]:
model.fit(X_train, y_train)

In [ ]:
evaluate(model,X_train, y_train, X_test, y_test)

## Итог:
Было построено несколько моделей для предсказания цены дома, в том числе с исользованием библиотеки XGBoost. Эту модель мы и попробуем загрузить как прототип:

In [ ]:
import pickle
with open('model.pkl','wb') as output:
    pickle.dump(model, output)